## Tools
https://langchain.readthedocs.io/en/latest/getting_started/getting_started.html

In [19]:
from dotenv import load_dotenv
import pprint

# Load the environment variables from .env
load_dotenv()

True

In [4]:
import os
"GOOGLE_API_KEY" in os.environ and "SERPAPI_API_KEY" in os.environ

True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
chat_groq = ChatGroq(temperature=0, model='llama3-groq-8b-8192-tool-use-preview')
llm = ChatGoogleGenerativeAI(temperature=0.9, model='gemini-pro')

d:\code\python\SETI\GenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load predefined tools

In [6]:
! pip install google-search-results

In [12]:
# Load in some tools to use
from langchain.agents import load_tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)


In [20]:
for tool in tools:
    print(tool)

name='Search' description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.' func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='0e06fba99dfce09b1ef7b14736248a801e4b6da97d58aeea528843c5b8ccb5d1', aiosession=None)> coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='0e06fba99dfce09b1ef7b14736248a801e4b6da97d58aeea528843c5b8ccb5d1', aiosession=None)>
name='Calculator' description='Useful for when you need to answer questions about math.' func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a

In [21]:
tools[0].invoke("Que hay para hacer en medellín? 3 sitios")


[{'block_position': 'top',
  'extensions': ['Free cancellation'],
  'extracted_price': 30,
  'link': 'https://www.getyourguide.com/',
  'position': 1,
  'price': '$30',
  'rating': 4.8,
  'reviews': 1400,
  'reviews_original': '1.4K',
  'source': 'GetYourGuide',
  'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTasqbabBkYIdajSgMxxnjrtVBLoZcRgmVsfM9h7H-RbBXb&s=4',
  'title': 'Medellin: Guatapé Tour, Lunch, Cruise, & Piedra del Peñol'},
 {'block_position': 'top',
  'extensions': ['Free cancellation'],
  'extracted_price': 63,
  'link': 'https://www.getyourguide.com/',
  'position': 2,
  'price': '$63',
  'rating': 4.8,
  'reviews': 102,
  'reviews_original': '102',
  'source': 'GetYourGuide',
  'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTyIocXnXTZaKaQJzYbMWNZnGAThDHAN4Zcwh1Z_X66hE0u&s=4',
  'title': 'Medellin: Coffee Tour and Wellness Spa'},
 {'block_position': 'top',
  'extensions': ['Free cancellation'],
  'extracted_price': 55,
  'link': 'h

In [16]:
tools[1].invoke("Que hay para hacer en medellín? 3 sitios")

ValueError: unknown format from LLM: This is not a math problem, so I cannot translate it into a mathematical expression.

In [22]:
llm_with_tools = llm.bind_tools(tools)
response1 = llm_with_tools.invoke("Quién es el actual presidente de Japón?")
print(response1)
print("Contenido:")
print(response1.content) #Vacío
print("Tool Calls:")
print(response1.tool_calls) # La invocación que requiere

AIMessage(content='', additional_kwargs={'function_call': {'name': 'Search', 'arguments': '{"__arg1": "Who is the current president of Japan?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f74ad4d1-853a-44f3-8f2f-6e69c5faa7b7-0', tool_calls=[{'name': 'Search', 'args': {'__arg1': 'Who is the current president of Japan?'}, 'id': '89057795-7169-40de-819c-eddc451b4a13', 'type': 'tool_call'}], usage_metadata={'input_tokens': 111, 'output_tokens': 21, 'total_tokens': 132})
Contenido:

Tool Calls:
[{'name': 'Search', 'args': {'__arg1': 

In [23]:
response2 = llm_with_tools.invoke("hola")
print("Contenido:")
print(response2.content) #Respuesta
print("Tool calls:")
print(response2.tool_calls) # vacio

Contenido:
I cannot fulfill this request. I do not understand Spanish.
Tool calls:
[]


### Create an Agent - Traditional method (initialize_agent)

In [10]:
# Finally, let's initialize an agent with:
# 1. The tools
# 2. The language model
# 3. The type of agent we want to use.
from langchain.agents import initialize_agent #Forma tradicional
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

d:\code\python\SETI\GenAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


## ReAct = Reasoning + Action

In [38]:
# Now let's test it out!
agent.run("Quién es el presidente de Bolivia? y cual es el mayor número par menor a su edad?")



> Entering new AgentExecutor chain...
I need to search about the president of Bolivia and calculate the largest even number smaller than his age.
Action: Search
Action Input: presidente de Bolivia
Observation: Luis Arce Catacora
Thought:The president of Bolivia is Luis Arce Catacora. Let us calculate the largest even number smaller than his age.
Action: Search
Action Input: Luis Arce Catacora edad
Observation: 60 years
Thought:The president of Bolivia is 60 years old, so the largest even number smaller than his age is 58.
Final Answer: 58

> Finished chain.


'58'

### Create an Agent - LangGraph Prebuilt

In [28]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

agent_executor = create_react_agent(llm, tools)

In [36]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})
for msg in response["messages"]:
    print(msg)

content='hi!' id='c1272e8c-10db-4421-b061-4691543eb828'
content='Hello there! I am a text-based AI assistant. How can I help you today?' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-db1527fe-ca4b-4b1a-b2e6-cad594ddfcda-0' usage_metadata={'input_tokens': 105, 'output_tokens': 19, 'total_tokens': 124}


In [39]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Quién es el presidente de Bolivia? y cuál es el mayor número par menor a su edad?")]})


In [40]:
for msg in response["messages"]:
    print(msg)

content='Quíen es el presidente de Bolivia? y cual es el mayor número par menor a su edad?' id='66225332-c98a-4d3b-966a-7ee6dbb07e7e'
content='' additional_kwargs={'function_call': {'name': 'Search', 'arguments': '{"__arg1": "presidente de Bolivia"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-607a8520-bec9-4461-9183-12dcd23a4d24-0' tool_calls=[{'name': 'Search', 'args': {'__arg1': 'presidente de Bolivia'}, 'id': '7fd76503-ae15-4690-916b-bb75cb8dbcbf', 'type': 'tool_call'}] usage_metadata={'input_tokens': 124, 'output_tokens': 16, 